## Introduction

In the business world, there is the most interesting question when people ask where should they open the business. It is the undeniable fact that the right place for business will always come with benefits such as more customers or more sales. It would be better if we use the customer generate data to decide the best place for a business instead of random choice and instinct. For example, we can find commonplace for restaurant type by using the data generated from Foursquare that defines the cluster by processing the code using python to show where in the neighborhood is restaurant cluster that customers will likely to go to find something to eat.

We will focus on Boston city as it is the largest city and the capital city of Massachusetts. There are also many people with a diversity of multinational.

In conclusion,we will try to use data from the Foursquare API to find the best neighborhood to open a restaurant in Boston city.

## Data

In this report, we will data from Boston city's neighborhood to solve our problem. our data for analysis come from 4 sources.

First from Analyze Boston website we use Boston Neighborhoods Datasets to obtain a list of Boston neighborhood.

Second from Statistical Atlas website for our neighborhood population density data.

Third, we use geocoder package to obtain Latitude and Longitude Data from the name of each neighborhood.

and the last is from Foursquare API which we use to obtain data such as venue in each neighborhood and category of it.

In [18]:
#import nessecary package
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import requests
import lxml.html as lh
import geocoder
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
#I have prepare and extract datasets already so let import it.

In [26]:
boston = pd.read_csv('/Users/kessawa/Desktop/bostont.csv',index_col=0)
#This is example of datasets that we will use except data from foursquare api that we will obtain during model building process.
boston

,Neighbourhood,Latitude,Longitude,Population Density
0,Roslindale,42.291209,-71.124497,9877.31
1,Jamaica Plain,42.309820,-71.120330,10363.86
2,Mission Hill,42.332926,-71.103214,22637.14
3,Longwood,42.336168,-71.099527,17561.48
4,Bay Village,42.350011,-71.066948,49840.22
5,Chinatown,42.352217,-71.062607,52322.61
6,North End,42.365097,-71.054495,42239.76
7,Roxbury,42.324843,-71.095016,14810.95
8,South End,42.341310,-71.077230,32320.73
9,Back Bay,42.350707,-71.079730,26282.87


In [27]:
#inspect stat from Population Density 
boston.iloc[:,-1:].describe()

,Population Density
count,24.000000
mean,20762.952917
std,15015.374429
min,1571.750000
25%,10242.222500
50%,15730.400000
75%,27792.335000
max,52322.610000


In [28]:

#retrieve boston lat and lng
address = 'Boston'

geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston are 42.3602534, -71.0582912.


In [29]:
# create map of Boston using latitude and longitude values
map_boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(boston['Latitude'], boston['Longitude'],boston['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

In [30]:
CLIENT_ID = 'FCUZKUDRBSQ21D2E1FNXQUAFYCSKFQTOJJ2VGGYXCQQBZI0J' # your Foursquare ID
CLIENT_SECRET = 'EWNJMXKI33YNYPHFN04ZJQIYCMZ2V4LFT2VFYJQX1NKADAW4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FCUZKUDRBSQ21D2E1FNXQUAFYCSKFQTOJJ2VGGYXCQQBZI0J
CLIENT_SECRET:EWNJMXKI33YNYPHFN04ZJQIYCMZ2V4LFT2VFYJQX1NKADAW4


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,Limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for x in venues_list for item in x])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
#get the venue information from each neighbourhood
boston_venues = getNearbyVenues(names=boston['Neighbourhood'],
                                   latitudes=boston['Latitude'],
                                   longitudes=boston['Longitude']
                                  )

Roslindale
Jamaica Plain
Mission Hill
Longwood
Bay Village
Chinatown
North End
Roxbury
South End
Back Bay
East Boston
Charlestown
West End
Beacon Hill
Downtown
Fenway
Brighton
West Roxbury
Hyde Park
Mattapan
Dorchester
South Boston
Allston
Harbor Islands


In [33]:
print(boston_venues.shape)

(959, 7)


In [34]:
print('There are {} uniques categories.'.format(len(boston_venues['Venue Category'].unique())))

There are 199 uniques categories.


In [35]:
#inspect stat from foursquare data
boston_venues.loc[:,['Venue','Venue Category']].describe(include='all')

,Venue,Venue Category
count,959,959
unique,780,199
top,Dunkin',Pizza Place
freq,16,50


In [36]:
boston_venues['Venue Category'].unique()[0:30]

array(['Scenic Lookout', 'Cuban Restaurant', 'Pizza Place',
       'American Restaurant', 'Pool', 'Big Box Store',
       'Rental Car Location', 'Donut Shop', 'Yoga Studio', 'Trail',
       'Coffee Shop', 'Accessories Store', 'Bookstore', 'Theater',
       'Liquor Store', 'Pet Store', 'Bakery', 'Deli / Bodega',
       'Seafood Restaurant', 'Art Gallery', 'Library', 'Noodle House',
       'Bar', 'Home Service', 'Park', 'Italian Restaurant',
       'Ice Cream Shop', 'Gastropub', 'Sandwich Place',
       'New American Restaurant'], dtype=object)

In [37]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighbourhood'] = boston_venues['Neighbourhood']

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + boston_onehot.columns[:-1].tolist()
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head(10)

,Neighbourhood,ATM,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tourist Information Center,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Roslindale,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,Jamaica Plain,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [38]:

print(boston_onehot.shape)

(959, 200)


In [39]:
#groupby neighbourhood and using mean function
boston_grouped = boston_onehot.groupby('Neighbourhood').mean().reset_index()
boston_grouped

,Neighbourhood,ATM,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tourist Information Center,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Allston,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.025974,0.000000,...,0.00,0.000000,0.000000,0.000000,0.038961,0.012987,0.000000,0.000000,0.000000,0.000000
1,Back Bay,0.000000,0.012987,0.064935,0.000000,0.000000,0.00,0.000000,0.000000,0.012987,...,0.00,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987
2,Bay Village,0.000000,0.000000,0.020833,0.000000,0.000000,0.00,0.020833,0.020833,0.000000,...,0.00,0.000000,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000,0.020833,0.000000
3,Beacon Hill,0.000000,0.000000,0.022222,0.000000,0.000000,0.00,0.000000,0.000000,0.022222,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222
4,Brighton,0.000000,0.000000,0.022727,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000
5,Charlestown,0.000000,0.000000,0.026316,0.000000,0.000000,0.00,0.000000,0.000000,0.026316,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316
6,Chinatown,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.000000,0.060000,0.000000,...,0.01,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.020000,0.000000,0.010000
7,Dorchester,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000
8,Downtown,0.000000,0.000000,0.027778,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778
9,East Boston,0.000000,0.000000,0.031250,0.000000,0.062500,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000


In [40]:

#display top 3 venue from each neighbourhood
num_top_venues = 3

for hood in boston_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                venue  freq
0   Korean Restaurant  0.08
1  Chinese Restaurant  0.04
2     Thai Restaurant  0.04


----Back Bay----
                 venue  freq
0          Coffee Shop  0.08
1  American Restaurant  0.06
2   Seafood Restaurant  0.05


----Bay Village----
                venue  freq
0  Seafood Restaurant  0.06
1      Sandwich Place  0.06
2              Bakery  0.06


----Beacon Hill----
                venue  freq
0         Pizza Place  0.09
1  Italian Restaurant  0.07
2               Plaza  0.04


----Brighton----
         venue  freq
0       Bakery  0.07
1  Pizza Place  0.07
2         Bank  0.07


----Charlestown----
               venue  freq
0        Pizza Place  0.08
1  Convenience Store  0.08
2        Coffee Shop  0.08


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.10
1              Bakery  0.08
2    Asian Restaurant  0.06


----Dorchester----
         venue  freq
0  Pizza Place  0.11
1          Gym  0.11
2        Plaza  0.11

In [41]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:

num_top_venues = 10

In [43]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = boston_grouped['Neighbourhood']

In [44]:

#create dataframe to display top common venues
for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

#preparing data for model
boston_grouped = pd.merge(boston_grouped,boston.iloc[:,[0,3]],how='outer',on=['Neighbourhood','Neighbourhood'])
boston_grouped_clustering = boston_grouped.drop('Neighbourhood', 1)
#drop neighbourhood that we didn't obtain data
boston_grouped_clustering.dropna(axis=0,inplace=True)

In [45]:

# set number of clusters
kclusters = 4
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 1, 1, 0, 2, 1, 0, 2, 2], dtype=int32)

In [47]:
# add clustering labels 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_ + 1)

In [48]:
boston_merged = boston

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [49]:

boston_merged.dropna(axis=0,inplace=True)
boston_merged.reset_index(inplace=True)
boston_merged.head()

,index,Neighbourhood,Latitude,Longitude,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Roslindale,42.291209,-71.124497,9877.31,3.0,Yoga Studio,Big Box Store,Cuban Restaurant,Pool,Rental Car Location,Scenic Lookout,Donut Shop,Pizza Place,American Restaurant,Hardware Store
1,1,Jamaica Plain,42.309820,-71.120330,10363.86,3.0,Bakery,Donut Shop,Art Gallery,Coffee Shop,Pizza Place,Library,Liquor Store,Noodle House,Deli / Bodega,Park
2,2,Mission Hill,42.332926,-71.103214,22637.14,1.0,Sandwich Place,Pizza Place,Sushi Restaurant,American Restaurant,Greek Restaurant,New American Restaurant,Coffee Shop,Gastropub,Donut Shop,Bar
3,3,Longwood,42.336168,-71.099527,17561.48,1.0,Donut Shop,Sandwich Place,Italian Restaurant,College Cafeteria,Gym,Gastropub,Coffee Shop,New American Restaurant,Bar,Pub
4,4,Bay Village,42.350011,-71.066948,49840.22,2.0,Spa,Seafood Restaurant,Bakery,Sandwich Place,Park,Italian Restaurant,Performing Arts Venue,Pizza Place,Theater,Burrito Place


In [50]:
#change type of cluster from float to int for easy inspect
boston_merged['Cluster Labels'] = boston_merged['Cluster Labels'].astype(int)

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['Latitude'], boston_merged['Longitude'], boston_merged['Neighbourhood'], boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1 ],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:

boston_merged.loc[boston_merged['Cluster Labels'] == 1,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Mission Hill,22637.14,1,Sandwich Place,Pizza Place,Sushi Restaurant,American Restaurant,Greek Restaurant,New American Restaurant,Coffee Shop,Gastropub,Donut Shop,Bar
3,Longwood,17561.48,1,Donut Shop,Sandwich Place,Italian Restaurant,College Cafeteria,Gym,Gastropub,Coffee Shop,New American Restaurant,Bar,Pub
7,Roxbury,14810.95,1,Park,Gym,Plaza,Art Gallery,Furniture / Home Store,Rental Car Location,Yoga Studio,Dive Bar,Farmers Market,Falafel Restaurant
12,West End,17420.70,1,Pizza Place,Donut Shop,Hotel,Sandwich Place,Bar,Italian Restaurant,Café,Coffee Shop,Sports Bar,Mexican Restaurant
16,Brighton,16433.47,1,Pizza Place,Bank,Bakery,Pub,Deli / Bodega,Coffee Shop,Bus Station,Grocery Store,Chinese Restaurant,Dance Studio
20,Dorchester,15027.33,1,Café,Shoe Store,Pizza Place,Southern / Soul Food Restaurant,Vegetarian / Vegan Restaurant,Plaza,Market,Fried Chicken Joint,Gym,Dog Run
22,Allston,19029.08,1,Korean Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Chinese Restaurant,Pizza Place,Bakery,Thai Restaurant,Italian Restaurant,Bubble Tea Shop,Pharmacy


In [55]:

boston_merged.loc[boston_merged['Cluster Labels'] == 2,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bay Village,49840.22,2,Spa,Seafood Restaurant,Bakery,Sandwich Place,Park,Italian Restaurant,Performing Arts Venue,Pizza Place,Theater,Burrito Place
5,Chinatown,52322.61,2,Chinese Restaurant,Bakery,Asian Restaurant,Sushi Restaurant,Coffee Shop,Theater,Sandwich Place,Seafood Restaurant,Hotel Bar,Pizza Place
6,North End,42239.76,2,Italian Restaurant,Seafood Restaurant,Pizza Place,Park,Bakery,Coffee Shop,Sports Bar,Sandwich Place,Spa,Café
13,Beacon Hill,47770.65,2,Pizza Place,Italian Restaurant,Plaza,French Restaurant,Hotel,Lake,Ice Cream Shop,Irish Pub,Kids Store,Korean Restaurant


In [56]:
boston_merged.loc[boston_merged['Cluster Labels'] == 3,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roslindale,9877.31,3,Yoga Studio,Big Box Store,Cuban Restaurant,Pool,Rental Car Location,Scenic Lookout,Donut Shop,Pizza Place,American Restaurant,Hardware Store
1,Jamaica Plain,10363.86,3,Bakery,Donut Shop,Art Gallery,Coffee Shop,Pizza Place,Library,Liquor Store,Noodle House,Deli / Bodega,Park
10,East Boston,8594.21,3,Pizza Place,Convenience Store,Art Gallery,Pharmacy,Latin American Restaurant,Sandwich Place,Bank,Fried Chicken Joint,Mexican Restaurant,Liquor Store
11,Charlestown,12192.04,3,Convenience Store,Coffee Shop,Pizza Place,Gastropub,Pub,Donut Shop,Yoga Studio,Plaza,Playground,History Museum
14,Downtown,5975.80,3,Playground,Pizza Place,Italian Restaurant,Food Truck,History Museum,Park,Other Repair Shop,Optical Shop,Mexican Restaurant,Mediterranean Restaurant
17,West Roxbury,5640.36,3,Home Service,Yoga Studio,Doctor's Office,Food Court,Food,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store,Dumpling Restaurant
18,Hyde Park,7474.90,3,American Restaurant,Pizza Place,ATM,Bank,Ice Cream Shop,Italian Restaurant,Gym,Grocery Store,Fast Food Restaurant,Donut Shop
19,Mattapan,12753.33,3,Pharmacy,Caribbean Restaurant,Home Service,Liquor Store,Pizza Place,Ice Cream Shop,Train Station,Cosmetics Shop,Bakery,Rental Car Location
21,South Boston,12192.04,3,Liquor Store,Pizza Place,Cosmetics Shop,Sports Bar,Bar,Coffee Shop,Italian Restaurant,Chinese Restaurant,Historic Site,Sandwich Place


In [57]:
boston_merged.loc[boston_merged['Cluster Labels'] == 4,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,South End,32320.73,4,Italian Restaurant,Coffee Shop,Park,Bar,Bakery,Wine Shop,French Restaurant,Gift Shop,Wine Bar,Dog Run
9,Back Bay,26282.87,4,Coffee Shop,American Restaurant,Seafood Restaurant,Ice Cream Shop,Italian Restaurant,Juice Bar,Gym / Fitness Center,Dessert Shop,Mediterranean Restaurant,Restaurant
15,Fenway,37978.28,4,Sports Bar,Pizza Place,Coffee Shop,Lounge,Furniture / Home Store,American Restaurant,Thai Restaurant,Baseball Field,Café,Donut Shop


## In conclude, the cluster show the most common venue in each Neighbourhood in the cluster which guild a person who want to open the business in each area cluster.